# Recomendaciones por lotes mediante el algoritmo de personalización del usuario de AWS y los datos de interacción entre el usuario y los elementos <a class="anchor" id="top"></a>

En este cuaderno, elegirá un conjunto de datos y lo preparará para utilizarlo con las recomendaciones por lote de Amazon Personalize.

1. [Elija un conjunto de datos o un origen de datos](#source)
1. [Prepare los datos](#prepare)
1. [Crear grupos de conjuntos de datos y el conjunto de datos de las interacciones](#group_dataset)
1. [Configurar un bucket de S3 y un rol de IAM](#bucket_role)
1. [Importar los datos de las interacciones](#import)
1. [Crear soluciones](#solutions)
1. [Recomendaciones por lote](#batch)
1. [Limpiar](#cleanup)

## Introducción <a class="anchor" id="intro"></a>

Por lo general, los algoritmos en Amazon Personalize (llamados recetas) buscan resolver diferentes tareas que se explican a continuación:

1. **Personalización del usuario de AWS**: recomienda elementos según las interacciones previas del usuario con los elementos.
1. **Personalized-Ranking**: toma una recopilación de elementos y, luego, los ubica en un posible orden de interés mediante un enfoque similar al de una HRNN.
1. **SIMS (elementos similares)**: en base a un elemento, recomienda otros elementos con los que también interactúan los usuarios.
1. **Popularity-Count**: recomienda los elementos más populares, si la HRNN o los metadatos de la HRNN no ofrecen una respuesta. Esta es la devolución predeterminada.

Sin importar cuál sea el caso de uso, todos los algoritmos comparten una base de aprendizaje sobre los datos de interacción entre el usuario y los elementos, la cual se define según 3 atributos principales:

1. **UserID**: el usuario que interactuó
1. **ItemID**: el elemento con el que interactuó el usuario
1. **Timestamp (Marca temporal)**: la hora a la que ocurrió la interacción

También admitimos los tipos de eventos y valores de eventos definidos por lo siguiente:

1. **Tipo de evento**: etiquetado de eventos por categoría (navegación, compra, clasificación, etc.).
1. **Valor de evento**: valor que corresponde al tipo de evento acontecido. En términos generales, buscamos valores normalizados entre 0 y 1 entre los tipos de eventos. Por ejemplo, si existen tres fases para completar una transacción (seleccionar, agregar al carrito y comprar), entonces hay un valor de evento para cada fase, como 0.33, 0.66 y 1.0, respectivamente.

Los campos para el tipo de evento y valor de evento son datos adicionales que se pueden utilizar para filtrar los datos enviados para el entrenamiento del modelo de personalización. En este ejercicio en particular no tendremos un tipo de evento ni un tipo de valor. 

## Elija un conjunto de datos o un origen de datos <a class="anchor" id="source"></a>
[Regresar al principio](#top)

Como ya lo mencionamos, los datos de la interacción del usuario con los elementos son cruciales para comenzar con el servicio. Esto significa que necesitamos buscar los casos de uso que generan ese tipo de datos, algunos ejemplos comunes son los siguientes:

1. Aplicaciones de video bajo demanda
1. Plataformas de comercio electrónico
1. Agregadores/plataformas de redes sociales

Existen algunas pautas para evaluar un problema que sea adecuado para Personalize. Como punto inicial, recomendamos los valores que figuran a continuación, aunque los [límites oficiales](https://docs.aws.amazon.com/personalize/latest/dg/limits.html) son un poco más bajos.

* Usuarios autenticados
* Al menos 50 usuarios exclusivos
* Al menos 100 elementos exclusivos
* Al menos 2 docenas de interacciones para cada usuario 

La mayoría de las veces esto es fácil de lograr, y si no llega a esta cantidad en alguna categoría, puede compensarlo con una mayor cantidad en otra categoría.

En general, Personalize no enviará los datos en forma perfecta y deberá realizar algunas modificaciones para que la estructura sea la correcta. Este cuaderno tiene como fin guiarlo en este proceso. 

Para comenzar, utilizaremos el conjunto de datos de [Last.FM](https://grouplens.org/datasets/hetrec-2011/). Estos son registros del comportamiento de escucha musical de los usuarios. Los datos son adecuados para nuestras pautas con una gran cantidad de usuarios, elementos e interacciones.

Primero, descargará el conjunto de datos y lo extraerá en una nueva carpeta por medio del código que aparece a continuación.

In [ ]:
data_dir = "data"
!mkdir $data_dir
!cd $data_dir && wget http://files.grouplens.org/datasets/hetrec2011/hetrec2011-lastfm-2k.zip
!cd $data_dir && unzip hetrec2011-lastfm-2k.zip

Observe los archivos de datos que ha descargado.

In [ ]:
!ls $data_dir

Actualmente, no se sabe mucho acerca de los datos, excepto que parece que hay muchos archivos .dat y un archivo README. Si abrimos el archivo README, este nos dará información acerca de la estructura general de estos datos. Este es un paso que quizá pueda omitir con datos personalizados, a menos que el origen de datos provenga de un equipo externo.

A partir del archivo README, podemos ver que existen varios tipos de interacciones en este conjunto de datos. Las interacciones entre usuarios que se definen como amigos entre ellos, las interacciones de usuarios que escuchan a artistas y las interacciones de etiquetas asignadas a los usuarios y artistas.

En este caso, nos enfocamos en los usuarios, los artistas y las interacciones de escucha. Tenemos 1892 usuarios, 17 632 artistas (nuestros elementos en este caso) y 92 834 interacciones de escucha de artistas por parte de usuarios. Esto es más que suficiente para que comencemos con Personalize.

Continúe leyendo atentamente el archivo README para llegar a la sección `Files`. La mayoría de los archivos en el conjunto de datos no son relevantes para nosotros, pero el archivo `users_artists.dat` se ve prometedor. La sección `Data format` del archivo README, proporciona más detalles sobre el contenido del archivo. Aquí es donde nos encontramos con el primer problema.

| userID | artistID | ponderación  |
|--------|----------|---------|
| 2      | 51       | 13883   |

Aunque existen datos de interacción entre los usuarios y los artistas que escuchan, estas interacciones quedan guardadas como ponderaciones, en lugar de marcas temporales. Para Amazon Personalize, necesitamos los datos de marcas temporales de la interacción entre el usuario y los elementos. 

Si observa nuevamente los archivos en el conjunto de datos, debería poder ver que `users_taggedartists-timestamps.dat` contiene datos de marca temporal. Entonces, ¿qué sucede si en lugar del comportamiento de escucha, utilizamos el comportamiento de etiquetado como nuestros datos de interacción? ¿Podemos suponer que si un usuario etiqueta a un artista esto indica un sentimiento positivo? Normalmente, lo discutiría con el cliente o alguien que tenga conocimiento de dominio para comprender si esta interacción es adecuada para el caso de uso que desea resolver. Por ahora, supondremos que el comportamiento de etiquetado es adecuado para nuestras necesidades. 

El esquema para `user_taggedartists-timestamps.dat` es:

| userID | artistID | tagID | timestamp (marca temporal)     |
|--------|----------|-------|---------------|
| 2      | 52       | 13    | 1238536800000 |

Si eliminamos el atributo `tagID`, tenemos exactamente el formato que necesitamos para Amazon Personalize.

## Prepare los datos <a class="anchor" id="prepare"></a>
[Regresar al principio](#top)

Lo que debe hacer a continuación es cargar los datos y confirmar que los datos se encuentran en buen estado. Luego, debe guardarlos en un archivo CSV donde estén listos para su uso con Amazon Personalize.

Para comenzar, importe una recopilación de bibliotecas de Python que se utilicen normalmente en la ciencia de datos.

In [ ]:
import time
from time import sleep
import json
from datetime import datetime

import boto3
import pandas as pd

Luego, abra el archivo de datos y observe varias de las primeras filas.

In [ ]:
original_data = pd.read_csv(data_dir + '/user_taggedartists-timestamps.dat')
original_data.head(5)

Evidentemente, los datos no se cargaron de forma correcta. El delimitador predeterminado para los archivos CSV (valores separados por comas) es una coma (`,`), pero en este caso el archivo se guardó con delimitadores de pestaña (`\t`). Entonces, especifiquemos el delimitador correcto e intentemos cargar los datos nuevamente.

In [ ]:
original_data = pd.read_csv(data_dir + '/user_taggedartists-timestamps.dat', delimiter='\t')
original_data.head(5)

Eso es mejor. Ahora que se han cargado los datos correctamente en la memoria, extraigamos información adicional. Primero, calculemos algunas estadísticas básicas a partir de los datos.

In [ ]:
original_data.describe()

Esto muestra que tenemos un buen rango de valores para `userID` y `artistID`. Luego, siempre es una buena idea confirmar el formato de los datos.

In [ ]:
original_data.info()

En base a esto, puede ver que hay un total de 186 479 entradas en el conjunto de datos, con 4 columnas y cada celda almacenada en formato int64.

El formato int64 es claramente adecuado para `userID` y `artistID`. Sin embargo, necesitamos examinar más a fondo para comprender las marcas temporales en los datos. Para utilizar Amazon Personalize, necesita guardar las marcas temporales en el formato [Unix Epoch (época de Unix)](https://en.wikipedia.org/wiki/Unix_time).

Actualmente, los valores de las marcas temporales no son legibles para el ser humano. Así que tomemos un valor de marca temporal arbitrario y resolvamos cómo interpretarlo.

In [ ]:
arb_time_stamp = original_data.iloc[50]['timestamp']
print(arb_time_stamp)
print(datetime.utcfromtimestamp(arb_time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

¡Ups! Para este valor de marca temporal en particular, el código generó el año 41 132. Eso es un futuro algo lejano para nosotros, por lo que claramente esta no era la forma correcta de analizar los datos. Necesitamos hacer un segundo intento.

JavaScript registra el tiempo en milisegundos y esta es una recopilación de datos de una aplicación web, así que dividamos el valor de la marca temporal por 1000 antes de aplicar nuestro código.

In [ ]:
arb_time_stamp = arb_time_stamp/1000
print(datetime.utcfromtimestamp(arb_time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

Febrero de 2009 se siente mucho más realista para nuestro conjunto de datos. Aunque no necesitamos marcas temporales legibles para humanos para utilizar Amazon Personalize, sí queremos que las fechas sean realistas, así que ahora transforme cada marca temporal en un conjunto de datos alejado del formato de milisegundos de JavaScript. 

In [ ]:
original_data.timestamp = original_data.timestamp / 1000
original_data.head(5)

Haga una comprobación de validez rápida en el conjunto de datos transformado mediante la elección de una marca temporal arbitraria y su transformación a un formato legible para humanos.

In [ ]:
arb_time_stamp = original_data.iloc[50]['timestamp']
print(arb_time_stamp)
print(datetime.utcfromtimestamp(arb_time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

Estos datos son coherentes como una marca temporal, para que podamos continuar formateando el resto de los datos. Recuerde que los datos que necesitamos son datos de interacción entre el usuario y los elementos, que serían `userID`, `artistID` y `timestamp` en este caso. Nuestro conjunto de datos tiene una columna adicional, `tagID`, la cual se puede eliminar del conjunto de datos.

In [ ]:
interactions_df = original_data.copy()
interactions_df = interactions_df[['userID', 'artistID', 'timestamp']]
interactions_df.head()

Luego de manipular los datos, siempre confirme si el formato de datos ha sido modificado.

In [ ]:
interactions_df.dtypes

En este caso, la columna de marca temporal pasó de int64 a float64. Entonces cambiemos el formato de vuelta a int64.

In [ ]:
interactions_df.astype({'timestamp': 'int64'}).dtypes

 Amazon Personalize tiene nombres de columnas predeterminados para los usuarios, los elementos y las marcas temporales. Estos nombres de columnas predeterminados son `USER_ID`, `ITEM_ID` Y `TIMESTAMP`. Por lo que la última modificación del conjunto de datos es reemplazar la columna de encabezados existente por los encabezados predeterminados.

In [ ]:
interactions_df.rename(columns = {'userID':'USER_ID', 'artistID':'ITEM_ID', 
                              'timestamp':'TIMESTAMP'}, inplace = True) 

¡Eso es todo! A esta altura, los datos están listos y solo necesitamos guardarlos como un archivo CSV.

In [ ]:
interactions_filename = "interactions.csv"
interactions_df.to_csv((data_dir+"/"+interactions_filename), index=False, float_format='%.0f')

## Crear grupos de conjuntos de datos y el conjunto de datos de las interacciones <a class="anchor" id="group_dataset"></a>
[Regresar al principio](#top)

El nivel más alto de aislamiento y abstracción con Amazon Personalize es un *grupo de conjuntos de datos*. La información almacenada dentro de uno de estos grupos de conjunto de datos no afecta ninguno de los demás grupos de conjuntos de datos o modelos creados de uno de ellos; están completamente aislados. Esto permite ejecutar muchos experimentos y es parte de lo que hacemos para mantener a los modelos de forma privada y completamente entrenados en sus datos. 

Antes de importar los datos que preparó antes, debe haber un grupo de conjuntos de datos y un conjunto de datos agregados a este que maneje las interacciones.

Los grupos de conjuntos de datos pueden almacenar los siguientes tipos de información:

* Interacciones entre el usuario y el elemento
* Flujo de eventos (interacciones en tiempo real)
* Metadatos del usuario
* Metadatos del elemento

Antes de crear el grupo de conjuntos de datos y el conjunto de datos para nuestros datos de interacción, validemos que su entorno se pueda comunicar exitosamente con Amazon Personalize.

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

### Crear el grupo de conjuntos de datos

La siguiente celda creará un nuevo grupo de conjuntos de datos con el nombre `personalize-poc-lastfm`.

In [ ]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "personalize-batch-recommendations-dg"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

Antes de que podamos utilizar el grupo de conjuntos de datos, este debe estar activo. Esto puede llevar uno o dos minutos. Ejecute la celda a continuación y aguarde hasta que muestre el estado ACTIVO. Esto verifica el estado del grupo de conjuntos de datos a cada segundo, hasta por un máximo de 3 horas.

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

Ahora que tiene un grupo de conjuntos de datos, puede crear un conjunto de datos para los datos de interacción.

### Crear el conjunto de datos

Primero, defina un esquema para indicar a Amazon Personalize el tipo de conjunto de datos que está cargando. Existen varias palabras clave reservadas y obligatorias, necesarias para el esquema, según el tipo de conjunto de datos. Para obtener más información, consulte la [documentación](https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html).

Aquí, creará un esquema para datos de interacciones, para lo que se necesitan los campos `USER_ID`, `ITEM_ID` y `TIMESTAMP`. En el esquema, estos deben estar definidos en el mismo orden que aparecen el conjunto de datos.

In [ ]:
interactions_schema = schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-batch-recommendations-interactions",
    schema = json.dumps(interactions_schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

Con un esquema creado, puede crear un conjunto de datos dentro del grupo de conjuntos de datos. Nota: Esto aún no carga los datos. Esto sucederá unos pasos más adelante.

In [ ]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-batch-recommendations-ds",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

## Configurar un bucket de S3 y un rol de IAM <a class="anchor" id="bucket_role"></a>
[Regresar al principio](#top)

Hasta ahora, hemos descargado, manipulado y guardado los datos en la instancia de Amazon EBS adjunta a la instancia que ejecuta este cuaderno de Jupyter. Sin embargo, Amazon Personalize necesitará un bucket de S3 para actuar como origen de los datos, así como roles de IAM para acceder a ese bucket. Preparemos todo esto.

Utilice los metadatos almacenados en la instancia subyacente a este cuaderno de Amazon SageMaker para determinar la región en la que opera. Si utiliza un cuaderno de Jupyter fuera de Amazon SageMaker, simplemente defina la región como una cadena a continuación. El bucket de Amazon S3 debe estar en la misma región que los recursos de Amazon Personalize que hemos creado hasta ahora.

In [ ]:
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print(region)

Los nombres de los buckets de Amazon S3 son exclusivos a nivel global. Para crear un nombre de bucket exclusivo, el siguiente código agregará la cadena `personalizepoc` a su número de cuenta de AWS. Luego, crea un bucket con este nombre en la región descubierta en la celda anterior.

In [ ]:
s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket_name = account_id + "-personalize-batch-recommendations"
print(bucket_name)
if region != "us-east-1":
    s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region})
else:
    s3.create_bucket(Bucket=bucket_name)

### Cargar los datos a S3

Ahora que creó su bucket de Amazon S3, cargue el archivo CSV de nuestros datos de interacción entre el usuario y los elementos. 

In [ ]:
interactions_file_path = data_dir + "/" + interactions_filename
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_filename).upload_file(interactions_file_path)
interactions_s3DataPath = "s3://"+bucket_name+"/"+interactions_filename

### Establecer la política del bucket de S3
Amazon Personalize debe ser capaz de leer el contenido de su bucket de S3. Así que agregue una política de bucket que lo permita.

In [ ]:
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:*Object",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

### Crear un rol de IAM

Amazon Personalize necesita la capacidad de asumir roles en AWS de modo que tenga los permisos para ejecutar ciertas tareas. Creemos un rol de IAM y adjuntemos las políticas necesarias a este. El siguiente código adjunta políticas muy permisivas. Utilice políticas más restrictivas para cualquier aplicación en producción.

In [ ]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleBatchRecommendations"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

## Importar los datos de las interacciones <a class="anchor" id="import"></a>
[Regresar al principio](#top)

Anteriormente, creó el grupo de conjuntos de datos y el conjunto de datos para alojar su información. Ahora ejecutará un trabajo de importación que cargará los datos del bucket de S3 en el conjunto de datos de Amazon Personalize. 

In [ ]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-batch-recommendations",
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

Antes de que podamos utilizar el grupo de conjuntos de datos, el trabajo de importación debe estar activo. Ejecute la celda a continuación y aguarde hasta que muestre el estado ACTIVO. Esto verifica el estado del trabajo de importación a cada segundo, por hasta un máximo de 3 horas.

La importación de los datos puede llevar algo de tiempo, según el tamaño del conjunto de datos. En este taller, el trabajo de importación debería tomar alrededor de 15 minutos.

In [ ]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

Cuando la importación del conjunto de datos está activa, está listo para comenzar a crear modelos con SIMS, clasificaciones personalizadas, recuentos de popularidad y personalización de usuario de AWS. Este proceso continuará en otros cuadernos. Ejecute la celda a continuación antes de pasar a almacenar algunos valores para su uso en los siguientes cuadernos.

## Crear soluciones <a class="anchor" id="solutions"></a>
[Regresar al principio](#top)

En este cuaderno, creará soluciones con las siguientes recetas:

1. Personalización del usuario de AWS


Una variación específica de un algoritmo se denomina receta en Amazon Personalize. Las diferentes recetas son adecuadas para diferentes situaciones. Un modelo entrenado se denomina solución, y cada solución puede tener muchas versiones relacionadas con un determinado volumen de datos cuando se entrenó el modelo.

Para comenzar, enumeraremos todas las recetas que son compatibles. Esto le permitirá seleccionar una y utilizarla para crear el modelo.

In [ ]:
personalize.list_recipes()

El resultado es solo una representación JSON de todos los algoritmos mencionados en la introducción.

A continuación, seleccionaremos recetas específicas y crearemos modelos con ellas.

### Personalización del usuario de AWS

La personalización de usuario de AWS es uno de los modelos de recomendación más avanzados que puede utilizar y que permite actualizaciones en tiempo real de recomendaciones basadas en el comportamiento del usuario. También tiende a conseguir mejores resultados que otros enfoques, como el filtrado colaborativo. La receta requiere el entrenamiento más largo, así que comencemos con esta receta primero.

Para nuestro caso de uso, mediante los datos de LastFM, podemos utilizar el algoritmo de personalización del usuario de AWS para recomendar nuevos artistas a un usuario en base al comportamiento previo de etiquetado de artistas de un usuario. Recuerde que hemos utilizado datos de etiquetas para representar interacciones positivas entre un usuario y un artista.

En primer lugar, seleccione la receta buscando el ARN en la lista de recetas anterior.

In [ ]:
recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization"

#### Crear la solución

Primero cree una solución con la receta. Aunque en este paso se proporciona el conjunto de datos ARN, el modelo aún no está entrenado. Véalo como un identificador en lugar de un modelo entrenado.

In [ ]:
create_solution_response = personalize.create_solution(
    name = "personalize-batch-recommendations-user-personalization",
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

#### Crear la versión de la solución

Una vez que tenga una solución, deberá crear una versión a fin de completar el entrenamiento del modelo. Completar el entrenamiento puede llevar un tiempo, más de 25 minutos, y un promedio de 40 minutos para esta receta con nuestro conjunto de datos. Por lo general, utilizaríamos un bucle while para hacer un seguimiento hasta que se complete la tarea. Sin embargo, la tarea bloquearía la ejecución de otras celdas, y el objetivo aquí es crear muchos modelos e implementarlos rápidamente. Así que estableceremos el bucle while para todas las soluciones más adelante en el cuaderno. Allí, también encontrará instrucciones para ver el progreso en la consola de AWS.

In [ ]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

In [ ]:
solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

### Ver el estado de creación de la solución

Según lo prometido, veremos cómo se pueden observar las actualizaciones de estado en la consola:

* En otra pestaña del navegador, ya debería tener la consola de AWS activa desde la apertura de esta instancia de cuaderno. 
* Cambie a esa pestaña y busque en la parte superior el servicio `Personalize`, luego diríjase a la página de ese servicio. 
* Haga clic en `View dataset groups` (Ver grupos de conjuntos de datos).
* Haga clic en el nombre de su grupo de conjuntos de datos, probablemente contenga POC en el nombre.
* Haga clic en `Solutions and recipes` (Ver grupos de conjuntos de datos).
* Ahora verá una lista de todas las soluciones que creó anteriormente, incluida una columna con el estado de las versiones de la solución. Una vez que esté `Active` (ACTIVA), su solución estará lista para la revisión. También es capaz de implementarse.

O simplemente ejecute la celda a continuación para seguir el estado de creación de la versión de la solución.

In [ ]:
in_progress_solution_versions = [
    solution_version_arn,
]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for solution_version_arn in in_progress_solution_versions:
        version_response = personalize.describe_solution_version(
            solutionVersionArn = solution_version_arn
        )
        status = version_response["solutionVersion"]["status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
    
    if len(in_progress_solution_versions) <= 0:
        break
    else:
        print("At least one solution build is still in progress")
        
    time.sleep(60)

## Interacción con las soluciones <a class="anchor" id="interact"></a>
[Regresar al principio](#top)

Ahora que todas las soluciones se han implementado, podemos comenzar a recibir recomendaciones a través de una llamada de API. 

Comience descargando el conjunto de datos que podemos utilizar para nuestra tabla de búsqueda.

In [ ]:
# Create a dataframe for the items by reading in the correct source CSV
items_df = pd.read_csv(data_dir + '/artists.dat', delimiter='\t', index_col=0)

# Render some sample data
items_df.head(5)

Cuando definimos la columna ID como una columna índice, es común que devuelva un artista con tan solo buscar el ID.

In [ ]:
item_id_example = 987
artist = items_df.loc[item_id_example]['name']
print(artist)

Eso no es malo, pero se podría volver complicado tener que repetir esto en cada parte de nuestro código, por lo que la siguiente función facilitará esto.

In [ ]:
def get_artist_by_id(artist_id, artist_df=items_df):
    """
    This takes in an artist_id from Personalize so it will be a string,
    converts it to an int, and then does a lookup in a default or specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return artist_df.loc[int(artist_id)]['name']
    except:
        return "Error obtaining artist"

Ahora probemos con algunos valores simples para verificar nuestra detección de errores.

In [ ]:
# A known good id
print(get_artist_by_id(artist_id="987"))
# A bad type of value
print(get_artist_by_id(artist_id="987.9393939"))
# Really bad values
print(get_artist_by_id(artist_id="Steve"))

¡Genial! Ahora tenemos una forma de conseguir resultados. 

## Recomendaciones por lote <a class="anchor" id="batch"></a>
[Regresar al principio](#top)

Existen muchos casos en los que quizá desee tener un mayor conjunto de datos de recomendaciones exportadas. Recientemente, Amazon Personalize lanzó recomendaciones por lote como una forma de exportar una colección de recomendaciones a S3. En este ejemplo, le explicaremos cómo hacer esto para la solución de personalización del usuario de AWS. Para obtener más información sobre las recomendaciones por lote, consulte la [documentación](https://docs.aws.amazon.com/personalize/latest/dg/getting-recommendations.html#recommendations-batch). Esta característica se aplica a todas las recetas, pero el formato de salida variará.

Una implementación sencilla tiene la siguiente apariencia:

```python
import boto3

personalize_rec = boto3.client(service_name='personalize')

personalize_rec.create_batch_inference_job (
    solutionVersionArn = "Solution version ARN",
    jobName = "Batch job name",
    roleArn = "IAM role ARN",
    jobInput = 
       {"s3DataSource": {"path": S3 input path}},
    jobOutput = 
       {"s3DataDestination": {"path":S3 output path"}}
)
```

Se ha determinado la importación SDK, la versión de la solución de arn y los roles de arn. Esto solo deja una entrada, una salida y un nombre de trabajo por definir.

Comencemos con la entrada para la Personalización del usuario que tiene la siguiente apariencia:


```JSON
{"userId": "4638"}
{"userId": "663"}
{"userId": "3384"}
```

Esto debería generar una salida con la siguiente apariencia:

```JSON
{"input":{"userId":"4638"}, "output": {"recommendedItems": ["296", "1", "260", "318"]}}
{"input":{"userId":"663"}, "output": {"recommendedItems": ["1393", "3793", "2701", "3826"]}}
{"input":{"userId":"3384"}, "output": {"recommendedItems": ["8368", "5989", "40815", "48780"]}}
```

La salida es un archivo JSON Lines. Consiste en objetos JSON individuales, uno por línea. Así que luego necesitaremos trabajar más para asimilar los resultados en este formato.

### Creación del archivo de entrada

Cuando utiliza la característica de lote, usted especifica los usuarios que desea que reciban recomendaciones para el momento en el que se haya completado el trabajo. La celda que aparece a continuación seleccionará nuevamente y de forma aleatoria a algunos usuarios y, luego, creará el archivo y lo guardará en el disco. A partir de allí, lo cargará al S3 para utilizarlo luego en la llamada a la API.

In [ ]:
users_df = pd.read_csv(data_dir + '/user_artists.dat', delimiter='\t', index_col=0)
# Render some sample data
users_df.head(5)

In [ ]:
# Get the user list
batch_users = users_df.sample(3).index.tolist()

# Write the file to disk
json_input_filename = "json_input.json"
with open(data_dir + "/" + json_input_filename, 'w') as json_input:
    for user_id in batch_users:
        json_input.write('{"userId": "' + str(user_id) + '"}\n')

In [ ]:
# Showcase the input file:
!cat $data_dir"/"$json_input_filename

Cargue el archivo a S3 y guarde el trayecto como una variable para más tarde.

In [ ]:
# Upload files to S3
boto3.Session().resource('s3').Bucket(bucket_name).Object(json_input_filename).upload_file(data_dir+"/"+json_input_filename)
s3_input_path = "s3://" + bucket_name + "/" + json_input_filename
print(s3_input_path)

Las recomendaciones por lote leen la entrada del archivo que hemos subido al S3. De forma similar, las recomendaciones por lote guardarán el archivo de salida en el S3. Así que definimos el trayecto de salida donde se deberían guardar los resultados.

In [ ]:
# Define the output path
s3_output_path = "s3://" + bucket_name + "/"
print(s3_output_path)

Ahora solo hagamos la llamada para poner en marcha el proceso de exportación del lote.

In [ ]:
batchInferenceJobArn = personalize.create_batch_inference_job (
    solutionVersionArn = solution_version_arn,
    jobName = "Batch-Inference-Job",
    roleArn = role_arn,
    jobInput = 
     {"s3DataSource": {"path": s3_input_path}},
    jobOutput = 
     {"s3DataDestination":{"path": s3_output_path}}
)
batchInferenceJobArn = batchInferenceJobArn['batchInferenceJobArn']

Ejecute el bucle while a continuación para hacer un seguimiento del estado de la llamada de recomendación por lote. Esto puede llevar alrededor de 25 minutos en completarse debido a que Personalize necesita soportar la infraestructura para llevar a cabo la tarea. Estamos probando la característica con un conjunto de datos de solo 3 usuarios, lo que no es un uso eficiente de este mecanismo. Normalmente, solo utilizaría esta característica para el procesamiento en masa, caso en el que las eficiencias se volverán claras.

In [ ]:
current_time = datetime.now()
print("Import Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_inference_job_response = personalize.describe_batch_inference_job(
        batchInferenceJobArn = batchInferenceJobArn
    )
    status = describe_dataset_inference_job_response["batchInferenceJob"]['status']
    print("DatasetInferenceJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.now()
print("Import Completed on: ", current_time.strftime("%I:%M:%S %p"))

Una vez que se han terminado de procesar las recomendaciones por lote, podemos tomar el resultado cargado a S3 y analizarlo.

In [ ]:
s3 = boto3.client('s3')
export_name = json_input_filename + ".out"
s3.download_file(bucket_name, export_name, data_dir+"/"+export_name)

# Update DF rendering
pd.set_option('display.max_rows', 30)
with open(data_dir+"/"+export_name) as json_file:
    # Get the first line and parse it
    line = json.loads(json_file.readline())
    # Do the same for the other lines
    while line:
        # extract the user ID 
        col_header = "User: " + line['input']['userId']
        # Create a list for all the artists
        recommendation_list = []
        # Add all the entries
        for item in line['output']['recommendedItems']:
            artist = get_artist_by_id(item)
            recommendation_list.append(artist)
        if 'bulk_recommendations_df' in locals():
            new_rec_DF = pd.DataFrame(recommendation_list, columns = [col_header])
            bulk_recommendations_df = bulk_recommendations_df.join(new_rec_DF)
        else:
            bulk_recommendations_df = pd.DataFrame(recommendation_list, columns=[col_header])
        try:
            line = json.loads(json_file.readline())
        except:
            line = None
bulk_recommendations_df

## Limpiar soluciones

Luego, limpie las soluciones. El código que aparece a continuación enumerará todas las soluciones en la cuenta.

In [ ]:
paginator = personalize.get_paginator('list_solutions')
for paginate_result in paginator.paginate():
    for solution in paginate_result["solutions"]:
        print(solution["solutionArn"])

Revise la lista de los ARN para determinar cuál solución desea eliminar. Luego, utilice el código que aparece a continuación para eliminarlo insertando el ARN.

In [ ]:
personalize.delete_solution(
    solutionArn = "INSERT ARN HERE"
)

## Limpiar conjuntos de datos

Luego, limpie los conjuntos de datos. El código que aparece a continuación enumerará todos los conjuntos de datos en la cuenta.

In [ ]:
paginator = personalize.get_paginator('list_datasets')
for paginate_result in paginator.paginate():
    for datasets in paginate_result["datasets"]:
        print(datasets["datasetArn"])

Revise la lista de los ARN para determinar qué conjunto de datos desea eliminar. Luego, utilice el código que aparece a continuación para eliminarlo insertando el ARN.

In [ ]:
personalize.delete_dataset(
    datasetArn = "INSERT ARN HERE"
)

## Limpiar esquemas

Luego, limpie los esquemas. El código que aparece a continuación enumerará todos los esquemas en la cuenta.

In [ ]:
paginator = personalize.get_paginator('list_schemas')
for paginate_result in paginator.paginate():
    for schema in paginate_result["schemas"]:
        print(schema["schemaArn"])

Revise la lista de los ARN para determinar qué esquema desea eliminar. Luego, utilice el código que aparece a continuación para eliminarlo insertando el ARN.

In [ ]:
personalize.delete_schema(
    schemaArn = "INSERT ARN HERE"
)

## Limpiar grupos de conjuntos de datos

Por último, limpie el grupo de conjuntos de datos. El código que aparece a continuación enumerará todos los grupos de conjuntos de datos en la cuenta.

In [ ]:
paginator = personalize.get_paginator('list_dataset_groups')
for paginate_result in paginator.paginate():
    for dataset_group in paginate_result["datasetGroups"]:
        print(dataset_group["datasetGroupArn"])

Revise la lista de los ARN para determinar qué grupo de conjuntos de datos desea eliminar. Luego, utilice el código que aparece a continuación para eliminarlo insertando el ARN.

In [ ]:
personalize.delete_dataset_group(
    datasetGroupArn = "INSERT ARN HERE"
)

## Limpiar bucket de S3 y rol de IAM

De forma opcional, puede eliminar el rol de IAM y el bucket de S3 que creamos a lo largo del curso.

Comience por enumerar todos los roles de IAM en la cuenta con el código que aparece a continuación.

In [ ]:
iam = boto3.client('iam')

paginator = iam.get_paginator('list_roles')
for paginate_result in paginator.paginate():
    for roles in paginate_result["Roles"]:
        print(roles["RoleName"])

Identifique el nombre del rol que desea eliminar.

No puede eliminar un rol de IAM que aún tiene políticas adjuntas. Después de que identifique el rol relevante, enumere las políticas adjuntas a ese rol.

In [ ]:
iam.list_attached_role_policies(
    RoleName = "INSERT ROLE NAME HERE"
)

Debe desconectar las políticas en el resultado anterior mediante el código que aparece a continuación. Repita este paso para cada política adjunta.

In [ ]:
iam.detach_role_policy(
    RoleName = "INSERT ROLE NAME HERE",
    PolicyArn = "INSERT ARN HERE"
)

Por último, debería poder eliminar el rol de IAM.

In [ ]:
iam.delete_role(
    RoleName = "INSERT ROLE NAME HERE"
)

Para eliminar un bucket de S3, primero debe estar vacío. La forma más sencilla de eliminar un bucket de S3 es simplemente navegar a S3 en la consola de AWS, eliminar los objetos en el bucket y, luego, eliminar el bucket de S3.